In [4]:
#AB Testi ile Bidding Yöntemlerinin Dönüşümünün Karşılaştırılması

In [6]:
#İş Problemi
#Facebook kısa süre önce mevcut "maximum bidding" adı verilen teklif verme türüne alternatif olarak yeni bir teklif türü olan 
#"average bidding"’i tanıttı. Müşterilerimizden biri olan bombabomba.com, bu yeni özelliği test 
#etmeye karar verdi ve average bidding'in maximum bidding'den daha fazla dönüşüm getirip getirmediğini anlamak için bir A/B 
#testi yapmak istiyor. A/B testi 1 aydır devam ediyor ve bombabomba.com şimdi sizden 
#bu A/B testinin sonuçlarını analiz etmenizi bekliyor. Bombabomba.com için nihai başarı ölçütü Purchase'dır. 
#Bu nedenle, istatistiksel testler için Purchase metriğine odaklanılmalıdır.

#Veri Seti Hikayesi

#Bir firmanın web site bilgilerini içeren bu veri setinde kullanıcıların gördükleri ve tıkladıkları reklam sayıları gibi bilgilerin yanı sıra 
#buradan gelen kazanç bilgileri yer almaktadır. Kontrol ve Test grubu olmak üzere iki ayrı veri seti vardır. Bu veri setleri
#ab_testing.xlsx excel’inin ayrı sayfalarında yer almaktadır. Kontrol grubuna Maximum Bidding, test grubuna Average Bidding uygulanmıştır.

In [8]:
pip install statsmodels

Note: you may need to restart the kernel to use updated packages.


In [29]:
#Görev 1: Veriyi Hazırlama ve Analiz Etme
#Adım 1: ab_testing_data.xlsx adlı kontrol ve test grubu verilerinden oluşan veri setini okutunuz. Kontrol ve test grubu verilerini ayrı 
#değişkenlere atayınız.
#Adım 2: Kontrol ve test grubu verilerini analiz ediniz.
#Adım 3: Analiz işleminden sonra concat metodunu kullanarak kontrol ve test grubu verilerini birleştiriniz.

In [9]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, pearsonr, spearmanr, kendalltau, f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest

In [10]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [13]:
#Veriyi excel'den okuma
df_control = pd.read_excel(r"C:\Users\ABRA\OneDrive\Belgeler\abtesting.xlsx", sheet_name="Control Group")
df_test = pd.read_excel(r"C:\Users\ABRA\OneDrive\Belgeler\abtesting.xlsx", sheet_name="Test Group")

In [15]:
df_control.head(5)

,Impression,Click,Purchase,Earning,Group
0,82529.45927,6090.07732,665.21125,2311.27714,Control
1,98050.45193,3382.86179,315.08489,1742.80686,Control
2,82696.02355,4167.96575,458.08374,1797.82745,Control
3,109914.40040,4910.88224,487.09077,1696.22918,Control
4,108457.76263,5987.65581,441.03405,1543.72018,Control


In [16]:
df_test.head(5)

,Impression,Click,Purchase,Earning,Group
0,120103.50380,3216.54796,702.16035,1939.61124,Test
1,134775.94336,3635.08242,834.05429,2929.40582,Test
2,107806.62079,3057.14356,422.93426,2526.24488,Test
3,116445.27553,4650.47391,429.03353,2281.42857,Test
4,145082.51684,5201.38772,749.86044,2781.69752,Test


In [17]:
df_control.describe().T

,count,mean,std,min,25%,50%,75%,max
Impression,40.00000,101711.44907,20302.15786,45475.94296,85726.69035,99790.70108,115212.81654,147539.33633
Click,40.00000,5100.65737,1329.98550,2189.75316,4124.30413,5001.22060,5923.80360,7959.12507
Purchase,40.00000,550.89406,134.10820,267.02894,470.09553,531.20631,637.95709,801.79502
Earning,40.00000,1908.56830,302.91778,1253.98952,1685.84720,1975.16052,2119.80278,2497.29522


In [18]:
df_test.describe().T

,count,mean,std,min,25%,50%,75%,max
Impression,40.00000,120512.41176,18807.44871,79033.83492,112691.97077,119291.30077,132050.57893,158605.92048
Click,40.00000,3967.54976,923.09507,1836.62986,3376.81902,3931.35980,4660.49791,6019.69508
Purchase,40.00000,582.10610,161.15251,311.62952,444.62683,551.35573,699.86236,889.91046
Earning,40.00000,2514.89073,282.73085,1939.61124,2280.53743,2544.66611,2761.54540,3171.48971


In [19]:
df_control["Group"] = "Control"
df_test["Group"] = "Test"

In [20]:
df_control["Group"]

0     Control
1     Control
2     Control
3     Control
4     Control
       ...   
35    Control
36    Control
37    Control
38    Control
39    Control
Name: Group, Length: 40, dtype: object

In [21]:
df_test["Group"]

0     Test
1     Test
2     Test
3     Test
4     Test
      ... 
35    Test
36    Test
37    Test
38    Test
39    Test
Name: Group, Length: 40, dtype: object

In [23]:
df = pd.concat([df_control, df_test], axis=0,ignore_index=False)
df.head(5)

,Impression,Click,Purchase,Earning,Group
0,82529.45927,6090.07732,665.21125,2311.27714,Control
1,98050.45193,3382.86179,315.08489,1742.80686,Control
2,82696.02355,4167.96575,458.08374,1797.82745,Control
3,109914.40040,4910.88224,487.09077,1696.22918,Control
4,108457.76263,5987.65581,441.03405,1543.72018,Control


In [24]:
df.groupby("Group")["Purchase","Click","Impression","Earning"].agg(["mean","min","max","sum","median"]).T

C:\Users\ABRA\AppData\Local\Temp\ipykernel_17528\3212792838.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df.groupby("Group")["Purchase","Click","Impression","Earning"].agg(["mean","min","max","sum","median"]).T


Group               Control         Test
Purchase mean     550.89406    582.10610
         min      267.02894    311.62952
         max      801.79502    889.91046
         sum    22035.76235  23284.24387
         median   531.20631    551.35573
...                     ...          ...
Earning  mean    1908.56830   2514.89073
         min     1253.98952   1939.61124
         max     2497.29522   3171.48971
         sum    76342.73199 100595.62931
         median  1975.16052   2544.66611

[20 rows x 2 columns]

In [30]:
#Görev 2: A/B Testinin Hipotezinin Tanımlanması
#Adım 1: Hipotezi tanımlayınız.
#H0 : M1 = M2 
#H1 : M1!= M2 
#Adım 2: Kontrol ve test grubu için purchase (kazanç) ortalamalarını analiz ediniz.

In [ ]:
#Görev 3: Hipotez Testinin Gerçekleştirilmesi
#Adım 1: Hipotez testi yapılmadan önce varsayım kontrollerini yapınız.
#Bunlar Normallik Varsayımı ve Varyans Homojenliğidir. Kontrol ve test grubunun normallik varsayımına uyup uymadığını Purchase değişkeni 
#üzerinden ayrı ayrı test ediniz.
#Normallik Varsayımı :
#H0: Normal dağılım varsayımı sağlanmaktadır. 
#H1: Normal dağılım varsayımı sağlanmamaktadır.
#p < 0.05 H0 RED , p > 0.05 H0 REDDEDİLEMEZ
#Test sonucuna göre normallik varsayımı kontrol ve test grupları için sağlanıyor mu ? Elde edilen p-value değerlerini yorumlayınız. 
#Varyans Homojenliği :
#H0: Varyanslar homojendir.
#H1: Varyanslar homojen Değildir.
#p < 0.05 H0 RED , p > 0.05 H0 REDDEDİLEMEZ
#Kontrol ve test grubu için varyans homojenliğinin sağlanıp sağlanmadığını Purchase değişkeni üzerinden test ediniz. 
#Test sonucuna göre normallik varsayımı sağlanıyor mu? Elde edilen p-value değerlerini yorumlayınız. 
#Adım 2: Normallik Varsayımı ve Varyans Homojenliği sonuçlarına göre uygun testi seçiniz.
#Adım 3: Test sonucunda elde edilen p_value değerini göz önünde bulundurarak kontrol ve test grubu satın alma ortalamaları arasında istatistiki 
#olarak anlamlı bir fark olup olmadığını yorumlayınız.

In [25]:
def AB_Test(dataframe, group, target):

    # Necessary packages
    from scipy.stats import shapiro
    import scipy.stats as stats

    # # Split A/B
    control = dataframe[dataframe[group] == "Control"][target] #Old Design
    test = dataframe[dataframe[group] == "Test"][target] #New Desing

    # Assumption of the Normality 
    normality_control = shapiro(control)[1] < 0.05
    normality_test = shapiro(test)[1] < 0.05

    # H0: Data follow a normal distribution.- False
    # H1: Data do not follow a normal distribution. - True

    if (normality_control == False) & (normality_test == False):  # "H0: Data follow a normal distribution
        # Parametric Test
        # Assumption: Homogeneity of variances

        leveneTest = stats.levene(control, test)[1] < 0.05
        # H0: Homogeneity: False
        # H1: Heterogeneous: True

        if leveneTest == False:
            # Homogeneity
            ttest = stats.ttest_ind(control, test, equal_var=True)[1] # Attention! equal_var=True
            # H0: M1 == M2 - False
            # H1: M1 != M2 - True
        else:
            # Heterogeneous
            ttest = stats.ttest_ind(control, test, equal_var=False)[1] #Attention! equal_var=False
            # H0: M1 == M2 - False
            # H1: M1 != M2 - True
    else:
        # Non-Parametric Test
        ttest = stats.mannwhitneyu(control, test)[1]
        # H0: M1 == M2 - False
        # H1: M1 != M2 - True

    # Result
    temp = pd.DataFrame({
        "AB Hypothesis": [ttest < 0.05],
        "p-value": [ttest]
    })
    temp["Test Type"] = np.where((normality_control == False) & (normality_test == False), "Parametric", "Non-Parametric")
    temp["AB Hypothesis"] = np.where(temp["AB Hypothesis"] == False, "Fail to Reject H0", "Reject H0")
    temp["Comment"] = np.where(temp["AB Hypothesis"] == "Fail to Reject H0", "A/B groups are similar!",
                               "A/B groups are not similar!")

    # Columns
    if (normality_control == False) & (normality_test == False):
        temp["Homogeneity"] = np.where(leveneTest == False, "Yes", "No")
        temp = temp[["Test Type", "Homogeneity", "AB Hypothesis", "p-value", "Comment"]]
    else:
        temp = temp[["Test Type", "AB Hypothesis", "p-value", "Comment"]]

    # Print Hypothesis
    print("# A/B Testing Hypothesis")
    print("H0: A == B")
    print("H1: A != B", "\n")

    return temp

In [26]:
# for purchase
AB_Test(dataframe=df, group="Group", target="Purchase")

# A/B Testing Hypothesis
H0: A == B
H1: A != B 



,Test Type,Homogeneity,AB Hypothesis,p-value,Comment
0,Parametric,Yes,Fail to Reject H0,0.34933,A/B groups are similar!


In [31]:
# A/B Testing Hypothesis
#H0: A == B
#H1: A != B
#p:0.34933 > 0.05 H0 REDDEDİLEMEZ yani Normal dağılım varsayımı sağlanmaktadır.